### Log Anomaly Detection on BGL dataset using LogBERT model
This is a running example of an end-to-end workflow of Log Anomaly Detection on public dataset HDFS using the LogBERT model.

There are similar workflows on the BGL datasets using other anomaly detectors (like LSTM based one in `bgl_lstm_unsupervised_parsed_sequential.ipynb`). 

The actual workflow script is exactly identical in these cases, except in the LogBERT case we choose to skip the log-parsing step. This is simply done following past literature, but there are no restrictions from the LogAI library side. 


Also check out the other config files that in this directory that cater to other datasets (HDFS), or other experimental configs like (parsing/nonparsing based, sliding/session window based log partitioning, sequential/semantic log feature representations, supervised/unsupervised setting, LSTM/CNN/Transformer/BERT model). 

To use these different experimental configs, you only need to point to the correct config file and the same workflow code should work perfectly for those!

Only in case of changing the dataset (eg. from BGL to HDFS) you need to not only change the config.yaml file but also use the HDFSPreprocessor in the preprocessing step. Note that each custom dataset that are added should have its own Preprocessor class (which should inherit from logai.preproces.preprocessor.Preprocessor). 

For more complete explanations of each step of the workflow check out the `hdfs_lstm_unsupervised_parsed_sequential.ipynb` notebook instead.


In [1]:
import os 
from logai.applications.openset.anomaly_detection.openset_anomaly_detection_workflow import OpenSetADWorkflowConfig, validate_config_dict
from logai.utils.file_utils import read_file
from logai.utils.dataset_utils import split_train_dev_test_for_anomaly_detection
import logging 
from logai.dataloader.data_loader import FileDataLoader
from logai.preprocess.bgl_preprocessor import BGLPreprocessor
from logai.information_extraction.log_parser import LogParser
from logai.preprocess.openset_partitioner import OpenSetPartitioner
from logai.analysis.nn_anomaly_detector import NNAnomalyDetector
from logai.information_extraction.log_vectorizer import LogVectorizer
from logai.utils import constants

In [2]:
config_path = "configs/bgl_logbert_config.yaml"
config_parsed = read_file(config_path)
config_dict = config_parsed["workflow_config"]
config = OpenSetADWorkflowConfig.from_dict(config_dict)

In [3]:
dataloader = FileDataLoader(config.data_loader_config)
logrecord = dataloader.load_data()
print (logrecord.body[constants.LOGLINE_NAME])

0         RAS KERNEL INFO instruction cache parity error...
1         RAS KERNEL INFO instruction cache parity error...
2         RAS KERNEL INFO instruction cache parity error...
3         RAS KERNEL INFO instruction cache parity error...
4         RAS KERNEL INFO instruction cache parity error...
                                ...                        
358455    RAS KERNEL FATAL idoproxy communication failur...
358456    RAS KERNEL FATAL idoproxy communication failur...
358457    RAS KERNEL FATAL idoproxy communication failur...
358458    RAS KERNEL FATAL idoproxy communication failur...
358459    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 358460, dtype: object


/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/logai/dataloader/data_loader.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected[constants.LOG_TIMESTAMPS] = pd.to_datetime(


In [4]:
preprocessor = BGLPreprocessor(config.preprocessor_config)
preprocessed_filepath = os.path.join(config.output_dir, 'BGL_11k_processed.csv')            
logrecord = preprocessor.clean_log(logrecord)
logrecord.save_to_csv(preprocessed_filepath)
print (logrecord.body[constants.LOGLINE_NAME])

0         RAS KERNEL INFO instruction cache parity error...
1         RAS KERNEL INFO instruction cache parity error...
2         RAS KERNEL INFO instruction cache parity error...
3         RAS KERNEL INFO instruction cache parity error...
4         RAS KERNEL INFO instruction cache parity error...
                                ...                        
358455    RAS KERNEL FATAL idoproxy communication failur...
358456    RAS KERNEL FATAL idoproxy communication failur...
358457    RAS KERNEL FATAL idoproxy communication failur...
358458    RAS KERNEL FATAL idoproxy communication failur...
358459    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 358460, dtype: object


In [5]:
partitioner = OpenSetPartitioner(config.open_set_partitioner_config)
partitioned_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session.csv')
logrecord = partitioner.partition(logrecord)
logrecord.save_to_csv(partitioned_filepath)
print (logrecord.body[constants.LOGLINE_NAME])

0       RAS KERNEL INFO instruction cache parity error...
1       RAS KERNEL INFO instruction cache parity error...
2       RAS KERNEL INFO instruction cache parity error...
3       RAS KERNEL INFO instruction cache parity error...
4       RAS KERNEL INFO instruction cache parity error...
                              ...                        
1848    RAS APP FATAL ciod Error reading message prefi...
1849    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1850    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1851    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1852    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 1853, dtype: object


In [6]:
train_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_unsupervised_train.csv')
dev_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_unsupervised_dev.csv')
test_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_unsupervised_test.csv')

(train_data, dev_data, test_data) = split_train_dev_test_for_anomaly_detection(
                logrecord,training_type=config.training_type,
                test_data_frac_neg_class=config.test_data_frac_neg,
                test_data_frac_pos_class=config.test_data_frac_pos,
                shuffle=config.train_test_shuffle
            )

train_data.save_to_csv(train_filepath)
dev_data.save_to_csv(dev_filepath)
test_data.save_to_csv(test_filepath)
print ('Train/Dev/Test Anomalous', len(train_data.labels[train_data.labels[constants.LABELS]==1]), 
                                   len(dev_data.labels[dev_data.labels[constants.LABELS]==1]), 
                                   len(test_data.labels[test_data.labels[constants.LABELS]==1]))
print ('Train/Dev/Test Normal', len(train_data.labels[train_data.labels[constants.LABELS]==0]), 
                                   len(dev_data.labels[dev_data.labels[constants.LABELS]==0]), 
                                   len(test_data.labels[test_data.labels[constants.LABELS]==0]))

indices_train/dev/test:  40 5 1808
Train/Dev/Test Anomalous 0 0 1808
Train/Dev/Test Normal 40 5 0


In [7]:
vectorizer = LogVectorizer(config.log_vectorizer_config)
vectorizer.fit(train_data)
train_features = vectorizer.transform(train_data)
dev_features = vectorizer.transform(dev_data)
test_features = vectorizer.transform(test_data)
print (train_features)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40
})


In [8]:
anomaly_detector = NNAnomalyDetector(config=config.nn_anomaly_detection_config)
anomaly_detector.fit(train_features, dev_features)

initialized data collator


The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 100
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the t

Step,Training Loss,Validation Loss
50,4.407800,7.452397
100,2.909800,3.440429


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 256
Saving model checkpoint to temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50
Configuration saved in temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50/config.json
Model weights saved in temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 256
Saving mod

In [9]:
predict_results = anomaly_detector.predict(test_features)
print (predict_results)

INFO:root:Loading model from /Users/amrita.saha/Home/salesforce/workspace/code/AIOps/RCA_Log/logai_opensource/logai/examples/jupyter_notebook/nn_ad_benchmarking/temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-100
loading configuration file /Users/amrita.saha/Home/salesforce/workspace/code/AIOps/RCA_Log/logai_opensource/logai/examples/jupyter_notebook/nn_ad_benchmarking/temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-100/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",


  0%|          | 0/1808 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 1849
  Batch size = 256
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


INFO:root:test_loss: 6.922546863555908 test_runtime: 390.1582 test_samples/s: 4.739
INFO:root:number of original test instances 1471
INFO:root:loss_mean Pos scores:  mean: 6.942281161443237, std: 0.774283953001779
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_max Pos scores:  mean: 8.484454650606613, std: 0.3959560440390038
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_top6_mean Pos scores:  mean: 7.668759094596368, std: 0.5011946579908295
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples

INFO:root:test_loss: 6.888496398925781 test_runtime: 403.7301 test_samples/s: 4.58
INFO:root:number of original test instances 1529
***** Running Prediction *****
  Num examples = 1849
  Batch size = 256


...

INFO:root:test_loss: 6.895295143127441 test_runtime: 389.3883 test_samples/s: 4.746
INFO:root:number of original test instances 1757
INFO:root:loss_mean Pos scores:  mean: 6.9251040041738445, std: 0.3305131315468509
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_max Pos scores:  mean: 8.789467540463061, std: 0.2957725925801199
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_top6_mean Pos scores:  mean: 7.770443531976949, std: 0.2481919072759722
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative sampl

INFO:root:test_loss: 6.904280662536621 test_runtime: 382.582 test_samples/s: 4.83
INFO:root:number of original test instances 1780
***** Running Prediction *****
  Num examples = 1848
  Batch size = 256


INFO:root:test_loss: 6.91431999206543 test_runtime: 383.5132 test_samples/s: 4.819
INFO:root:number of original test instances 1805
INFO:root:loss_mean Pos scores:  mean: 6.931001467315855, std: 0.3142642478823886
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_max Pos scores:  mean: 8.80874226641457, std: 0.29194871734675143
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
INFO:root:

INFO:root:loss_top6_mean Pos scores:  mean: 7.755777619240943, std: 0.26667536160998695
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative sample

INFO:root:test_loss: 6.865622043609619 test_runtime: 393.0448 test_samples/s: 4.702
INFO:root:number of original test instances 1808


      indices  max_loss   sum_loss num_loss  \
0           0  8.259521  57.445042        9   
1           0  8.235804  47.465973        8   
2           1  8.030133  62.417809        8   
3           2  8.638699  68.155144        9   
4           2  8.026514  62.823498        8   
...       ...       ...        ...      ...   
18478    1804  8.474268  63.616337        8   
18479    1805   8.47681  52.004642        8   
18480    1806   8.77824  55.250885        9   
18481    1807  8.028709  56.912701        9   
18482    1807  8.787713  63.224758        8   

                                               top6_loss  \
0      [8.259520530700684, 8.195176124572754, 7.94188...   
1      [8.235803604125977, 8.206790924072266, 7.84517...   
2      [8.030133247375488, 8.012803077697754, 7.98151...   
3      [8.63869857788086, 8.119754791259766, 7.925565...   
4      [8.026514053344727, 8.017280578613281, 7.98747...   
...                                                  ...   
18478  [8.47426